In [1]:
#Imports
import os, sys
path_to_package = os.path.abspath(os.path.join('../'))
if path_to_package not in sys.path:
    sys.path.append(path_to_package)


import matplotlib.pyplot as plt
import pandas as pd
import json

In [2]:
phenotypes = [
    "SM300-Efficiency", "SM300-Rate", "SM300-Lag", "SM300-AUC",
    "SM60-Efficiency",  "SM60-Rate",  "SM60-Lag",  "SM60-AUC",
    "Ratio-Efficiency", "Ratio-Rate", "Ratio-Lag", "Ratio-AUC",
]

models = ["ridge", "lasso", 'rf']

EXP_NUM = 1

In [36]:
def get_original_position(exp_num, index_data, model_pos):
    if model_pos > len(index_data['anova'][str(exp_num)]):
        print(index_data["filename"])
        print("exp", exp_num)
        print(model_pos)
        print(sorted(index_data['anova'][str(exp_num)]))
        print(len(index_data['anova'][str(exp_num)]))
        
    anova_index = index_data['anova'][str(exp_num)][model_pos]
    clean_index = index_data['clean']['index'][anova_index]
    # print(f"anova: {anova_index} | clean: {clean_index}")
    return clean_index


def load_index(filename):
    index_path = "/home/khaldrem/code/sc_regmod/dataset/index"
    f = open(f"{index_path}/{filename}.json", "r")
    json_data = json.load(f)
    f.close()

    return json_data
    
def create_resume_exp_coef(models, phenotypes, exp_num, top_n_features=15):
    save_path = f"res/{exp_num}"
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for model in models:
        final_data = {}

        print(f"model: {model}")
        for pt in phenotypes:
            # print(pt)
            # print()
            path = f"/home/khaldrem/code/sc_regmod/dataset/models/final/{exp_num}/csv/{model}_coef_results_{pt}.csv"
            all_data = pd.read_csv(path)

            data = all_data.iloc[:top_n_features, :]
            filenames = data["filename"].tolist()
            res_data = []
            positions = []

            if model == "rf":
                res_data = data["absolute_error"].tolist()
                positions = data["feature"].tolist()
            else:
                res_data = data["coefficients"].tolist()
                positions = data["features"].tolist()

            
            positions = [int(i.split("_")[1].split("x")[1]) for i in positions]
            
            original_position = []
            for fn, pos in zip(filenames, positions):
                # print(fn)
                # print(pos)
                index_data = load_index(fn)                
                original_position.append(get_original_position(exp_num, index_data, pos))

            final_data[f'{pt}_filenames'] = filenames
            final_data[f'{pt}_importance'] = res_data
            final_data[f'{pt}_original_index'] = original_position

        final_data_df = pd.DataFrame.from_dict(final_data)
        final_data_df.to_csv(f'{save_path}/{model}.csv')
        

        

In [47]:
create_resume_exp_coef(models, phenotypes, 1, 50)

model: ridge
model: lasso
model: rf


In [48]:
create_resume_exp_coef(models, phenotypes, 2, 50)

model: ridge
model: lasso
model: rf


In [49]:
create_resume_exp_coef(models, phenotypes, 3, 50)

model: ridge
model: lasso
model: rf


In [50]:
create_resume_exp_coef(models, phenotypes, 4, 50)

model: ridge
model: lasso
model: rf
